## 명목척도
### 단일표본
- 적합성 검정
    - 카이제곱분포 활용
    - from scipy.stats import chisquare
    - 사례 : 세 사람의 지지율은 0.05 유의수준에서 다르다고 할 수 있는가?
- Run 검정
    - 1표본 Run검정 : 한 개의 샘플이 무작위로 추출되었는지의 여부를 검정
        - from statsmodels.sandbox.stats.runs import runstest_1samp
    - 2표본 Run검정 : 두 개의 샘플이 하나의 모집단에서 왔는지 검정하라.
        - from statsmodels.sandbox.stats.runs import runstest_2samp
### 대응표본-2
- Mcnemar 검정 : 대응관계가 있는 이항변수 2개 데이터의 분포 차이
    - 이항변수이므로 1/0 집계를 한 후 table형태로 검정
    - 두 변수의 데이터 분포는 차이가 있다/없다.
    - from statsmodels.stats.contingency_tables import mcnemar
### 대응표본-k
- Cochran Q 검정 : 대응관계가 있는 이항변수 3개 이상 데이터의 분포 차이
    - 대응관계 일원배치 분산분석이 비모수 버전
    - 각 변수의 데이터분포 차이가 있다/없다.
    - from statsmodels.stats.contingency_tables import cochrans_q
### 독립표본
- 독립성 검정
    - 교차분석
    - from scipy.stats import fisher_exact
    - 사례 : 성별 변량과 안경착용여부 변량이 서로 독립인지 5% 유의수준에서 검정하라.
- 동질성 검정
    - from scipy.stats import chi2_contingency
    - 사례 : 프로그램 A, B, C에 대해 연령층 별로 선호도가 다른지 5% 유의수준에서 검정하라.

## 서열척도
### 단일표본
- 부호검정
    - 부호검정이므로 주어진 상황을 초과/미초과, 해당/미해당, 차이가 있음/없음 식으로 상황을 변환한다.
        - +부호의 개수는 이항분포를 따른다.
    - 1표본 부호검정 (1표본 t검정의 비모수 버전)
        - n<=100 이면 이항분포를 따르고, n>100이면 정규분포를 따른다.
- 부호순위 검정
    - 1표본 윌콕슨 부호순위 검정 (1표본 t검정의 비모수 버전)
        - n<=이면 윌콕슨 순위합 분포를 따르고 n>20이면 정규분포에 근사한다.
        - from scipy.stats import wilconxon
- 부호순위검정
### 대응표본-2
- 부호검정
    - 부호검정이므로 주어진 상황을 초과/미초과, 해당/미해당, 차이가 있음/없음 식으로 상황을 변환한다.
        - +부호의 개수는 이항분포를 따른다.
    - 2표본 부호검정 (대응표본 t검정의 비모수 버전)
        - n<=100 이면 이항분포를 따르고, n>100이면 정규분포를 따른다.
- 부호순위 검정
    - 2표본 윌콕슨 부호순위 검정 (대응표본 t검정의 비모수 버전)
        - n<=이면 윌콕슨 순위합 분포를 따르고 n>20이면 정규분포에 근사한다.
### 대응표본-k
- Friedman 검정
    - 세 변수 이상의 대응관계가 있는 데이터의 차이 검정
    - 대응표본 1원배치 분산분석으로 사후 검정은 윌콕슨 순위합 검정
    - (3개 범주 and 데이터크기 9이하) OR (4개범주 and 데이터크기가 5이하) 의 경우 프리드먼 검정표, 그렇지 않으면 카이제곱 분포
    - from scipy.stats import friedmanchisquare
### 독립표본-2
- 순위합검정 (독립표본 t검정에 대응)
    - n(n1+n2) <=25면 순위합분포, n>25면 정규분포에 근사
    - from scipy.stats import ranksums, mannwitneyu
- 만위트니-U 검정
### 독립표본-k
- Kruskal_Wallis 검정
    - 세 변수 이상의 대응관계가 없는 데이터의 차이 검증
    - 독립표본 1원배치 분산분석으로 사후검정은 윌콕슨 순위합 검정
    - (3개 범주 and 데이터크기 15이하) OR (4개범주 and 데이터크기가 14이하) 의 경우 프리드먼 검정표, 그렇지 않으면 카이제곱 분포
    - from scipy.stats import kruskal


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['axes.unicode_minus'] = False


### 1. 
#### A, B, C 동별 입주민의 주민대표 찬반 투표결과. 동별 찬반 비율이 동일한지 귀무가설, 대립가설 세우고 검정통계량 검정
- [찬성] - A:50, B:60, C:65
- [반대] - A:45, B:32, C:55

In [4]:
# 지금 이 상황을 어떤 상황으로 봐야 할까? A, B, C 3개의 찬/반 비율이 같은지 검정해야 한다. 이 3개의 값은 같은 값인가?
# 그러면 비모수검정: 명목척도: 독립표본 3: 동일성 검정으로 본다.

# H0 : A, B, C 3개 집단의 찬/반 비율 차이는 없다.
# H1 : A, B, C 3개 집단의 찬/반 비율 차이가 있다.

data = pd.DataFrame(data={'A':[50, 45], 'B':[60, 32], 'C':[65, 55]}, index=['찬성', '반대'])
data

from scipy.stats import chi2_contingency
chistat, p, df, expec = chi2_contingency(data, correction=False)

print('검정통계량은 {:.3f}이고 유의확률 p-value는 {:.3f} 이다.'.format(chistat, p))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 없다.'.format(p))

검정통계량은 3.667이고 유의확률 p-value는 0.160 이다.
유의확률이 0.160이므로 귀무가설을 기각할 수 없다.


### 2. 
#### 캠페인에 노출되기 전/후, 캠페인에 대한 지지 여부를 기록한 데이터. 전/후 지지율 차이가 있는지 귀무가설, 대립가설 설정. 검정통계량 계산

In [13]:
# 비모수검정 - 명목척도 - 대응표본 2개 (전/후) - Mcnemar 검정
# 적합성검정/Run검정 | [대응표본2] Mcneamar 검정 | [대응표본k]Cochran Q검정 | [독립표본]독립성검정/동질성검정

# H0 : 노출 전/후 캠페인에 대한 지지율 차이는 없다.
# H1 : 노출 전/후 캠페인에 대한 지지율 차이가 있다.

data = pd.read_csv('https://raw.githubusercontent.com/algoboni/pythoncodebook1-1/refs/heads/main/practice6_ba.csv', index_col=0)

from pandas import crosstab
ct = crosstab(data.before, data.after)
print(ct)

from statsmodels.stats.contingency_tables import mcnemar
mc = mcnemar(ct, exact=False, correction=False)     # 파라미터 exact 값이 True면 이항분포, False면 카이제곱 분포를 사용. correction=False는 카이제곱분포
print('검정통계량은 {:.3f} 유의확률은 {:.3f}'.format(mc.statistic, mc.pvalue))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 없다.'.format(mc.pvalue))

after   0   1
before       
0       4  12
1       9   6
검정통계량은 0.429 유의확률은 0.513
유의확률이 0.513이므로 귀무가설을 기각할 수 없다.


### 3. 
#### 다음 T/F가 무작위로 나열되어 있는지, 귀무가설/대립가설을 설정하고 검정통계량 계산하시오.
- 1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1

In [14]:
# 비모수검정 - 명목척도 - 단일샘플 - 적합성검정/Run검정
# H0 : 무작위 추출로 볼 수 있다.
# H1 : 무작위 추출로 볼 수 없다.

data = [1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1]

from statsmodels.sandbox.stats.runs import runstest_1samp
zstat, pval = runstest_1samp(data)

print('검정통계량은 {:.3f}, 유의확률은 {:.3f}입니다.'.format(zstat, pval))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 있습니다.'.format(pval))


검정통계량은 2.187, 유의확률은 0.029입니다.
유의확률이 0.029이므로 귀무가설을 기각할 수 없습니다.


### 4. 
#### 연봉상승률이 20대보다 30대가 높다는 주장. 타당성 확인위해 20대 10명, 30대 8명 임의추출. 기대연봉상승율 조사
- [20대] 3.0, 3.5, 2.0, 2.8, 5.0, 0, 2.3, 2.8, 3.3, 3.5
- [30대] 3.5, 5.5, 5.0, 5.0, 10.0, 8.0, 2.5, 3.0

In [38]:
# 간단하게만 생각하면 20대의 평균과 30대의 평균만 비교하면 될 일이다. 
# 이런 경우 두 표본이 같은 모집단으로부터 나왔는지를 검정하기 위해, 대응표본의 경우 t검정을 시행하기도 한다.
# 그런데 이 경우는 둘이 독립표본이다. 두 독립표본 간의 차이가 있는지를 보겠다는 것이다.

# 비모수검정 - 명목척도 - 2표본 Run검정으로 수행한다.

#H0 : 두 집단의 분포에 차이는 없다.
#H1 : 두 집단의 분포에 차이가 있다. 

data2 = [3.0, 3.5, 2.0, 2.8, 5.0, 0, 2.3, 2.8, 3.3, 3.5]
data3 = [3.5, 5.5, 5.0, 5.0, 10.0, 8.0, 2.5, 3.0]

n2, n3 = len(data2), len(data3)
m2, m3 = np.median(data2), np.median(data3)

tb2 = pd.DataFrame(data2, columns=['data'])
tb2['M'] = m2
tb3 = pd.DataFrame(data3, columns=['data'])
tb3['M'] = m3

tb = pd.concat([tb2, tb3])
tb['rank'] = tb.data.rank(method='average')

w2 = tb.query('M==@m2')['rank'].sum()
w3 = tb.query('M==@m3')['rank'].sum()

u2 = w2 - n2*(n2+1)/2
u3 = w3 - n3*(n3+1)/2

print('순위합 검정통계량 {:.3f} or {:.3f}'.format(w2, w3))
print('만위트니 검정통계량 {:.3f} or {:.3f}'.format(u2, u3))

from scipy.stats import mannwhitneyu
mstat, p = mannwhitneyu(data2, data3, alternative='less')

print('검정통계량은 {:.3f}, 유의확률은 {:.3f}이다.'.format(mstat, p))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 있다.'.format(p))

순위합 검정통계량 69.500 or 101.500
만위트니 검정통계량 14.500 or 65.500
검정통계량은 14.500, 유의확률은 0.013이다.
유의확률이 0.013이므로 귀무가설을 기각할 수 있다.


### 5. 
#### 양식/한식/중식에 대한 5명의 선호도. 비선호:1, 보통:2, 선호:3으로 코딩. 음식종류에 따라 선호도차이가 있는지 귀무가설, 대립가설 설정. 검정통계량
-          A,B,C,D,E
- [양식] 1, 3, 1, 3, 1
- [한식] 1, 3, 3, 1, 1
- [중식] 2, 3, 3, 3, 1

In [19]:
# 비모수검정: 서열척도: 대응표본 K: Friedmann 검정 
#   --> 5명의 3개의 수준에 대한 답변. 한 사람마다 3개의 수준에 대해 답변한 것. 따라서 대응표본
# H0 : 5명의 음식취향에는 차이가 없다.
# H1 : 5명의 음식취향에는 차이가 있다.

from scipy.stats import friedmanchisquare
table = pd.DataFrame([[1, 3, 1, 3, 1], [1, 3, 3, 1, 1], [2, 3, 3, 3, 1]], columns=['A', 'B','C','D','E'], index=['양식', '한식', '중식'])

# 넘겨줄 때 level 별로 넘겨준다. 
stat, p = friedmanchisquare(table.values[0], table.values[1], table.values[2])
print('검정통계량은 {:.3f}이고 유의확률은 {:.3f}이다.'.format(stat, p))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 없다.'.format(p))

검정통계량은 2.667이고 유의확률은 0.264이다.
유의확률이 0.264이므로 귀무가설을 기각할 수 없다.


### 6. 
#### A, B, C 지역 학생들의 키. 지역별로 학생 들의 키가 차이가 있는지 귀무가설, 대립가설 설정. 검정통계량 계산
- A = [177, 167, 188, 189, 152, 159, 184, 175]
- B = [151, 177, 150, 187, 167, 166, 179, 161, 174]
- C = [173, 151, 156, 182, 188, 175, 150, 165, 176, 183]

In [24]:
#서열/명목척도는 아니나 모수를 기반으로 추정/검정할 수 없거나, 할 필요가 없거나.
#독립표본 K : kruskal_wallis검정 혹은 독립성, 동질성 검정

A = [177, 167, 188, 189, 152, 159, 184, 175]
B = [151, 177, 150, 187, 167, 166, 179, 161, 174]
C = [173, 151, 156, 182, 188, 175, 150, 165, 176, 183]

from scipy.stats import kruskal
table = pd.DataFrame([A, B, C], index=['A', 'B', 'C'])

stat, p = kruskal(table.values[0], table.values[1], table.values[2], nan_policy='omit')

print('검정통계량은 {:.3f}, 유의확률은 {:.3f}'.format(stat, p))

검정통계량은 1.331, 유의확률은 0.514


## 명목척도
### 단일표본
- 적합성 검정
    - 카이제곱분포 활용
    - from scipy.stats import chisquare
    - 사례 : 세 사람의 지지율은 0.05 유의수준에서 다르다고 할 수 있는가?
- Run 검정
    - 1표본 Run검정 : 한 개의 샘플이 무작위로 추출되었는지의 여부를 검정
        - from statsmodels.sandbox.stats.runs import runstest_1samp
    - 2표본 Run검정 : 두 개의 샘플이 하나의 모집단에서 왔는지 검정하라.
        - from statsmodels.sandbox.stats.runs import runstest_2samp
### 대응표본-2
- Mcnemar 검정 : 대응관계가 있는 이항변수 2개 데이터의 분포 차이
    - 이항변수이므로 1/0 집계를 한 후 table형태로 검정
    - 두 변수의 데이터 분포는 차이가 있다/없다.
    - from statsmodels.stats.contingency_tables import mcnemar
### 대응표본-k
- Cochran Q 검정 : 대응관계가 있는 이항변수 3개 이상 데이터의 분포 차이
    - 대응관계 일원배치 분산분석이 비모수 버전
    - 각 변수의 데이터분포 차이가 있다/없다.
    - from statsmodels.stats.contingency_tables import cochrans_q
### 독립표본
- 독립성 검정
    - 교차분석
    - from scipy.stats import fisher_exact
    - 사례 : 성별 변량과 안경착용여부 변량이 서로 독립인지 5% 유의수준에서 검정하라.
- 동질성 검정
    - from scipy.stats import chi2_contingency
    - 사례 : 프로그램 A, B, C에 대해 연령층 별로 선호도가 다른지 5% 유의수준에서 검정하라.

## 서열척도
### 단일표본
- 부호검정
    - 부호검정이므로 주어진 상황을 초과/미초과, 해당/미해당, 차이가 있음/없음 식으로 상황을 변환한다.
        - +부호의 개수는 이항분포를 따른다.
    - 1표본 부호검정 (1표본 t검정의 비모수 버전)
        - n<=100 이면 이항분포를 따르고, n>100이면 정규분포를 따른다.
- 부호순위 검정
    - 1표본 윌콕슨 부호순위 검정 (1표본 t검정의 비모수 버전)
        - n<=이면 윌콕슨 순위합 분포를 따르고 n>20이면 정규분포에 근사한다.
        - from scipy.stats import wilconxon
- 부호순위검정
### 대응표본-2
- 부호검정
    - 부호검정이므로 주어진 상황을 초과/미초과, 해당/미해당, 차이가 있음/없음 식으로 상황을 변환한다.
        - +부호의 개수는 이항분포를 따른다.
    - 2표본 부호검정 (대응표본 t검정의 비모수 버전)
        - n<=100 이면 이항분포를 따르고, n>100이면 정규분포를 따른다.
- 부호순위 검정
    - 2표본 윌콕슨 부호순위 검정 (대응표본 t검정의 비모수 버전)
        - n<=이면 윌콕슨 순위합 분포를 따르고 n>20이면 정규분포에 근사한다.
### 대응표본-k
- Friedman 검정
    - 세 변수 이상의 대응관계가 있는 데이터의 차이 검정
    - 대응표본 1원배치 분산분석으로 사후 검정은 윌콕슨 순위합 검정
    - (3개 범주 and 데이터크기 9이하) OR (4개범주 and 데이터크기가 5이하) 의 경우 프리드먼 검정표, 그렇지 않으면 카이제곱 분포
    - from scipy.stats import friedmanchisquare
### 독립표본-2
- 순위합검정 (독립표본 t검정에 대응)
    - n(n1+n2) <=25면 순위합분포, n>25면 정규분포에 근사
    - from scipy.stats import ranksums, mannwitneyu
- 만위트니-U 검정
### 독립표본-k
- Kruskal_Wallis 검정
    - 세 변수 이상의 대응관계가 없는 데이터의 차이 검증
    - 독립표본 1원배치 분산분석으로 사후검정은 윌콕슨 순위합 검정
    - (3개 범주 and 데이터크기 15이하) OR (4개범주 and 데이터크기가 14이하) 의 경우 프리드먼 검정표, 그렇지 않으면 카이제곱 분포
    - from scipy.stats import kruskal


In [2]:
mat = [96,93,63,89,85,84,66,62,90]
eng = [98,90,74,84,69,69,73,61,98]

# --------------------------------------------------------------------
# 피어슨 상관계수 
# --------------------------------------------------------------------
from scipy.stats import pearsonr
corr, p = pearsonr(mat, eng)
print(corr, p)
# 유의수준 0.05


# 신뢰수준 90% 신뢰구간


# --------------------------------------------------------------------
# 스피어만  순위상관계수
# --------------------------------------------------------------------

0.7454392939220784 0.021141520899581358
